<a href="https://colab.research.google.com/github/jainnipun/MachineLearning/blob/master/Clustering/CustomerSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nipunucd","key":"b47ef5fe4b97e8886e4248a15c083c12"}'}

In [0]:

#Install the Kaggle API client.
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
#!kaggle datasets list

In [0]:
#!kaggle competitions download -c microsoft-malware-prediction

In [0]:
#!kaggle datasets download -d lava18/google-play-store-apps

In [8]:
!kaggle datasets download -d carrie1/ecommerce-data

 63% 5.00M/7.95M [00:00<00:00, 36.8MB/s]
100% 7.95M/7.95M [00:00<00:00, 50.9MB/s]


In [9]:
!unzip -d ecommerce ecommerce-data.zip

Archive:  ecommerce-data.zip
  inflating: ecommerce/data.csv      


In [10]:
!ls -a

.  ..  .config	ecommerce  ecommerce-data.zip  kaggle.json  sample_data


In [11]:
!ls ecommerce

data.csv


In [0]:
import pandas as pd
data=pd.read_csv('ecommerce/data.csv',encoding = "ISO-8859-1")

In [0]:
import datetime

data['InvoiceDate'] = data['InvoiceDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y %H:%M").strftime("%Y-%m-%d"))

In [14]:
print(data['InvoiceDate'].min())
print(data['InvoiceDate'].max())
max_date = data['InvoiceDate'].max()

2010-12-01
2011-12-09


In [0]:
data['Amount'] = data['Quantity'] * data['UnitPrice']

In [0]:
data=data.loc[data['CustomerID'].isnull()==False]

In [0]:
#data.loc[data['Amount']==0]

In [0]:
#Removing records where Quatity is 0

data = data.loc[data['Quantity']!=0]

In [0]:
#data.loc[data['CustomerID']==18274.0]
#data.loc[data['CustomerID'] == 18276.0]


In [0]:
#data.loc[data['CustomerID'] == 12347.0]


In [19]:
#Viewing nan records - if any present in data we will rectify that

nans = lambda df: df[df.isnull().any(axis=1)]
nans(data)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Amount


In [0]:
#Not remove data where amount is 0 as we will consider free purchases as well
#data = data[data.Amount != 0]

In [0]:
#processedData = data.groupby(by=['CustomerID'])

In [0]:
# Recency  - Last Transaction date
# Frequency - Total Tranactions
# Monetary - Total Spendings

processedData = pd.DataFrame(columns=['CustomerId', 'Recency', 'Frequency','Monetary'])

In [0]:
data_group_by_customers = data.groupby(by=['CustomerID'])

In [0]:
#data_group_by_customers.InvoiceNo.nunique()

In [0]:
#Counting unique invoice per customer
processedData['Frequency'] = data.groupby(by=['CustomerID']).InvoiceNo.nunique()

#Fetching last invoice date per customer
processedData['Recency']=data.groupby(by='CustomerID').max()['InvoiceDate']

#Fetching total spending per customer
processedData['Monetary'] =  data.groupby(by='CustomerID').sum()[['Amount']]

processedData['CustomerId']=(list(processedData.index.values)) 

In [0]:
#Removing negative records - where refund greater than purchase
processedData = processedData.loc[processedData['Monetary']>=0.0]


In [0]:
import datetime

now = datetime.datetime.now()

processedData['Recency']=processedData['Recency'].apply(lambda x:(now-datetime.datetime.strptime(x,'%Y-%m-%d')).days)


In [28]:
processedData

,CustomerId,Recency,Frequency,Monetary
CustomerID,,,,
12346.0,12346.0,2960,2,0.000000e+00
12347.0,12347.0,2637,7,4.310000e+03
12348.0,12348.0,2710,4,1.797240e+03
12349.0,12349.0,2653,1,1.757550e+03
12350.0,12350.0,2945,1,3.344000e+02
12352.0,12352.0,2671,11,1.545410e+03
12353.0,12353.0,2839,1,8.900000e+01
12354.0,12354.0,2867,1,1.079400e+03
12355.0,12355.0,2849,1,4.594000e+02
